# Real Time Speech to Translated Speech 

Translating my own voice into a different language, as a file.

---
## OpenAI For Translation


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "Key" # Add ur key

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

translation_template = """
Translate the following sentence into {language}, return ONLY the translation, nothing else.

Sentence: {sentence}
"""

output_parser = StrOutputParser()
llm = ChatOpenAI(temperature=0.0, model="gpt-4.1-mini")
translation_prompt = ChatPromptTemplate.from_template(translation_template)

translation_chain = (
    {"language": RunnablePassthrough(), "sentence": RunnablePassthrough()} 
    | translation_prompt
    | llm
    | output_parser
)

def translate(sentence, language="Telugu"):
    data_input = {"language": language, "sentence": sentence}
    translation = translation_chain.invoke(data_input)
    return translation

---
## ElevenLabs For Voice Cloning & Voice Synthesis

Premade voice model on [ElevenLabs Service](https://elevenlabs.io/app/voice-lab), using Multilingual V2 Model for synthesis

There are so many languages thats available. You can create a voice by simply giving a prompt.

In [ ]:
# Use this once for downloading required packages to run below code.
# !pip install elevenlabs 

In [ ]:
# from fileinput import filename
from elevenlabs.client import ElevenLabs

client = ElevenLabs(api_key="Key") # Add ur own key

def gen_dub(text: str, voice_id: str, filename: str ):
    print("Generating audio…")
    audio = client.text_to_speech.convert(
        text=text,
        voice_id=voice_id,
        model_id="eleven_multilingual_v2",
        output_format="mp3_44100_128",
    )

    with open(filename, "wb") as f: 
        for chunk in audio:
            f.write(chunk)

    print(f"Audio saved to {filename}")


---
## AssemblyAI for Speech to Text Streaming

AssemblyAI handles Streaming STT within their own platform. Inserting the above translation and voice generation functions within this workflow.

In [ ]:
# Use this once for downloading required packages to run below code.

# !pip install assemblyai


In [ ]:
import assemblyai as aai

aai.settings.api_key = "Key" # Replace with ur actual key


transcriber = aai.Transcriber()

def transcribe_file_and_dub(
    audio_filepath: str,
    target_language: str,
    voice_name: str 
):
    """
    1) Upload & transcribe the given audio file.
    2) Run translation on the transcript.
    3) Generate & play the dubbed audio.
    """
    print(f"Transcribing '{audio_filepath}'…")
    transcript = transcriber.transcribe(audio_filepath)
    
    if transcript.error:
        print("Transcription error:", transcript.error)
        return

    text = transcript.text
    print("Transcript:", text)

    print("Translating…")
    translation = translate(text, target_language)
    print("Translation:", translation)

    gen_dub(translation, voice_name, filename="dubbed_audio.mp3")


---
## Main Script

(remember to change audio input/output in settings for airpods)

In [ ]:
audio_path = "/Users/sharon/Downloads/acts/acts17(part2).mp3"  # or .wav/.m4a, etc.

transcribe_file_and_dub(audio_path, target_language="Telugu", voice_name="M26s8JE0fGtkluZqQKYP")